### https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=test.csv

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import sidetable
%matplotlib inline

df_train = pd.read_csv('./train.csv', index_col='Id')
X_train = df_train.drop(columns='SalePrice')
y_train = df_train['SalePrice']
X_test = pd.read_csv('./test.csv', index_col='Id')
train_test = pd.concat([X_train, X_test])
train_index = X_train.index
test_index = X_test.index

In [2]:
# Have a look at the first five rows
train_test.head()

MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
Id                                                                    
1           60       RL         65.0     8450   Pave   NaN      Reg   
2           20       RL         80.0     9600   Pave   NaN      Reg   
3           60       RL         68.0    11250   Pave   NaN      IR1   
4           70       RL         60.0     9550   Pave   NaN      IR1   
5           60       RL         84.0    14260   Pave   NaN      IR1   

   LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC Fence  \
Id                                  ...                                     
1          Lvl    AllPub    Inside  ...           0        0    NaN   NaN   
2          Lvl    AllPub       FR2  ...           0        0    NaN   NaN   
3          Lvl    AllPub    Inside  ...           0        0    NaN   NaN   
4          Lvl    AllPub    Corner  ...           0        0    NaN   NaN   
5          Lvl    AllPub       FR2  ...           0        0    NaN   NaN   

   MiscFeature MiscVal  MoSold  YrSold  SaleType  SaleCondition  
Id                                                               
1          NaN       0       2    2008        WD         Normal  
2          NaN       0       5    2007        WD         Normal  
3          NaN       0       9    2008        WD         Normal  
4          NaN       0       2    2006        WD        Abnorml  
5          NaN       0      12    2008        WD         Normal  

[5 rows x 79 columns]

In [3]:
threshold = 0.3 * len(train_test)
cols = train_test.isnull().sum()[train_test.isnull().sum() < threshold].index
train_test = train_test[cols]
X_train = train_test.loc[train_index, cols]
X_test = train_test.loc[test_index, cols]

In [4]:
from sklearn.impute import SimpleImputer
from category_encoders.cat_boost import CatBoostEncoder
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer

num_cols = train_test.select_dtypes(include='number').columns
cat_cols = train_test.select_dtypes(exclude='number').columns
si = SimpleImputer(strategy='most_frequent', fill_value='Unknown')
ce = CatBoostEncoder()
ii = IterativeImputer()
sc = StandardScaler()

X_train.drop_duplicates(inplace=True)
X_test.drop_duplicates(inplace=True)

si.fit(train_test[cat_cols])
X_train[cat_cols] = si.transform(X_train[cat_cols])
X_test[cat_cols] = si.transform(X_test[cat_cols])

ce.fit(X_train[cat_cols], y_train)
X_train[cat_cols] = ce.transform(X_train[cat_cols])
X_test[cat_cols] = ce.transform(X_test[cat_cols])

ii.fit(train_test[num_cols])
X_train[num_cols] = ii.transform(X_train[num_cols])
X_test[num_cols] = ii.transform(X_test[num_cols])

sc.fit(train_test[num_cols])
X_train[num_cols] = sc.transform(X_train[num_cols])
X_test[num_cols] = sc.transform(X_test[num_cols])